In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
import numpy as np

In [2]:
BATCH_SIZE = 128
EPOCHS = 50
LAYER_SIZE = 30
S_LAYER_SIZE = 15
LATENT_SIZE = 4

In [3]:
# Load the data from the file

saved = np.load("testing2.npy", allow_pickle=True)

# Make a trainging and testing batch
train_data = torch.Tensor(saved[:int(len(saved)*0.5)])
test_data = torch.Tensor(saved[int(len(saved)*0.5):])
print(train_data.shape)
INPUT_SIZE = len(test_data[0])
INPUT_SIZE

torch.Size([50000, 12])


12

In [4]:
device = "cuda"

In [5]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        # Encoder
        self.l1 = nn.Linear(INPUT_SIZE, LAYER_SIZE)
        self.ltest1 = nn.Linear(LAYER_SIZE, S_LAYER_SIZE)
        self.l2a = nn.Linear(S_LAYER_SIZE, LATENT_SIZE)
        self.l2b = nn.Linear(S_LAYER_SIZE, LATENT_SIZE)
        
        # Decoder
        self.l3 = nn.Linear(LATENT_SIZE, S_LAYER_SIZE)
        self.ltest2 = nn.Linear(S_LAYER_SIZE, LAYER_SIZE)
        self.l4 = nn.Linear(LAYER_SIZE, INPUT_SIZE)
            
    # Run some data through the encoder
    def encode(self, x):
        out = F.relu(self.l1(x))
        out = F.relu(self.ltest1(out))
        # return the mu and the sigma
        return self.l2a(out), self.l2b(out)
    
    # The reparameterization trick, taken from https://github.com/pytorch/examples/blob/master/vae/main.py
    def reparameterize(self, mu, sigma):
        std = torch.exp(0.5*sigma)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, x):
        out = F.relu(self.l3(x))
        out = F.relu(self.ltest2(out))
        return torch.sigmoid(self.l4(out)) # sigmoid vs tanh
    
    def forward(self, x):
        # TODO: check the shape of x to be sure we have the right input
        mu, sigma = self.encode(x)
        z = self.reparameterize(mu, sigma)
        # The loss function needs the mu and the sigma so just return them here
        return self.decode(z), mu, sigma

In [6]:
# Reconstruction + KL divergence losses summed over all elements and batch
# Taken from https://github.com/pytorch/examples/blob/master/vae/main.py
def loss_function(recon_x, x, mu, sigma):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + sigma - mu.pow(2) - sigma.exp())
    return BCE + KLD

In [7]:
def train(e_count, model, optimizer, name, data):
    model.train()
    train_loss = 0
    for i in range(0, len(data), BATCH_SIZE):
        batch = data[i:i+BATCH_SIZE].to(device)
        model.zero_grad()
        recons, mu, sigma = model(batch)
        loss = loss_function(recons, batch, mu, sigma)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
    print(f"Model: {name}, -- Epoch: {e_count}, Loss: {train_loss/len(data)}")        
        
    
    

In [8]:
# Lets use CPU
def test(e_count, model, optimizer):
    model.eval()
    test_loss = 0
    for i in range(0, len(test_data), BATCH_SIZE):
        batch = test_data[i:i+BATCH_SIZE].to(device)
        recons, mu, sigma = model(batch)
        loss = loss_function(recons, batch, mu, sigma)
        test_loss += loss.item()
    print(f"TEST Epoch: {e_count}, Loss: {test_loss/len(test_data)}") 
        
        

In [9]:
def get_data(data, balanced, size):
    res = []
    for d in data:
        if d[10] == int(balanced):
            res.append(d)
        if len(res) == size:
            break
    return torch.Tensor(res)

In [10]:
train_data=train_data[torch.randperm(train_data.size()[0])]


balanced = torch.cat((get_data(saved[:int(len(saved)*0.5)], True, 5000), get_data(saved[:int(len(saved)*0.5)], False, 5000)), 0)

In [11]:
len(balanced)

10000

In [12]:
balanced

tensor([[0.5003, 0.4994, 0.5108,  ..., 0.0500, 1.0000, 1.0000],
        [0.4998, 0.4998, 0.5004,  ..., 0.0500, 1.0000, 1.0000],
        [0.5003, 0.5005, 0.5002,  ..., 0.0500, 1.0000, 1.0000],
        ...,
        [0.5003, 0.5001, 0.5003,  ..., 0.0500, 0.0000, 0.0000],
        [0.4997, 0.5004, 0.5003,  ..., 0.0500, 0.0000, 0.0000],
        [0.4997, 0.5004, 0.5003,  ..., 0.0500, 0.0000, 0.0000]])

In [13]:
balanced=balanced[torch.randperm(balanced.size()[0])]
# SHUFFLE

In [14]:
balanced

tensor([[0.4996, 0.4998, 0.5000,  ..., 0.0500, 0.0000, 0.0000],
        [0.5001, 0.5001, 0.4996,  ..., 0.0500, 0.0000, 0.0000],
        [0.4999, 0.5001, 0.4997,  ..., 0.0500, 0.0000, 0.0000],
        ...,
        [0.4999, 0.5002, 0.4996,  ..., 0.0500, 1.0000, 1.0000],
        [0.5001, 0.4997, 0.4999,  ..., 0.0500, 0.0000, 0.0000],
        [0.5002, 0.4998, 0.4996,  ..., 0.0500, 0.0000, 0.0000]])

In [15]:
# TWO DATA SETS, TWO MODELS
train_data = train_data[:len(balanced)]

vae1 = VAE().to(device)
opt1 = optim.Adam(vae1.parameters(), lr=0.001)

vae2 = VAE().to(device)
opt2 = optim.Adam(vae2.parameters(), lr=0.001)

In [16]:
for i in range(EPOCHS):
    train(i, vae1,opt1, "unbalanced", train_data)

Model: unbalanced, -- Epoch: 0, Loss: 7.914503137207031
Model: unbalanced, -- Epoch: 1, Loss: 7.40919640045166
Model: unbalanced, -- Epoch: 2, Loss: 7.357793789672852
Model: unbalanced, -- Epoch: 3, Loss: 7.348363154602051
Model: unbalanced, -- Epoch: 4, Loss: 7.321791301727295
Model: unbalanced, -- Epoch: 5, Loss: 7.288438598632813
Model: unbalanced, -- Epoch: 6, Loss: 7.233796565246582
Model: unbalanced, -- Epoch: 7, Loss: 7.169334674072266
Model: unbalanced, -- Epoch: 8, Loss: 7.140749060821533
Model: unbalanced, -- Epoch: 9, Loss: 7.106517601776123
Model: unbalanced, -- Epoch: 10, Loss: 7.079235556793213
Model: unbalanced, -- Epoch: 11, Loss: 7.081819658660889
Model: unbalanced, -- Epoch: 12, Loss: 7.067290048217774
Model: unbalanced, -- Epoch: 13, Loss: 7.064394281768799
Model: unbalanced, -- Epoch: 14, Loss: 7.0729996223449705
Model: unbalanced, -- Epoch: 15, Loss: 7.055854309844971
Model: unbalanced, -- Epoch: 16, Loss: 7.059308688354492
Model: unbalanced, -- Epoch: 17, Loss: 7.

In [17]:
for i in range(EPOCHS):
    train(i, vae2,opt2, "BALANCED" ,balanced)

Model: BALANCED, -- Epoch: 0, Loss: 8.112312348937989
Model: BALANCED, -- Epoch: 1, Loss: 7.855789385986328
Model: BALANCED, -- Epoch: 2, Loss: 7.832775335693359
Model: BALANCED, -- Epoch: 3, Loss: 7.829481066131592
Model: BALANCED, -- Epoch: 4, Loss: 7.827793279266357
Model: BALANCED, -- Epoch: 5, Loss: 7.8244663360595705
Model: BALANCED, -- Epoch: 6, Loss: 7.795593988800049
Model: BALANCED, -- Epoch: 7, Loss: 7.631918147277832
Model: BALANCED, -- Epoch: 8, Loss: 7.5334119132995605
Model: BALANCED, -- Epoch: 9, Loss: 7.469767429351807
Model: BALANCED, -- Epoch: 10, Loss: 7.393831839752197
Model: BALANCED, -- Epoch: 11, Loss: 7.359883326721191
Model: BALANCED, -- Epoch: 12, Loss: 7.359900345611572
Model: BALANCED, -- Epoch: 13, Loss: 7.356234034729004
Model: BALANCED, -- Epoch: 14, Loss: 7.355520979309082
Model: BALANCED, -- Epoch: 15, Loss: 7.34039623413086
Model: BALANCED, -- Epoch: 16, Loss: 7.343116334533692
Model: BALANCED, -- Epoch: 17, Loss: 7.351442469787598
Model: BALANCED, --

In [18]:
# Shuffle test data
test_data=test_data[torch.randperm(test_data.size()[0])]

ex_done = None
ex_notdone = None
for a in test_data:
    if a[10] == 1:
        ex_done = a
        break

for a in test_data:
    if a[10] == 0:
        ex_notdone = a
        break

In [19]:
ex_done[10]

tensor(1.)

In [20]:
list(vae1(ex_done.to("cuda")))[0][10]

tensor(0.9674, device='cuda:0', grad_fn=<SelectBackward>)

In [21]:
list(vae2(ex_done.to("cuda")))[0][10]

tensor(0.9980, device='cuda:0', grad_fn=<SelectBackward>)

In [22]:
ex_notdone[10]

tensor(0.)

In [23]:
list(vae1(ex_notdone.to("cuda")))[0][10]

tensor(8.8627e-08, device='cuda:0', grad_fn=<SelectBackward>)

In [24]:
list(vae2(ex_notdone.to("cuda")))[0][10]

tensor(0.0098, device='cuda:0', grad_fn=<SelectBackward>)

In [25]:
count = 0
for d in balanced:
    if d[10] == 1:
        count += 1
print((count/len(train_data))*100)

50.0


In [26]:
count = 0
for d in train_data:
    if d[10] == 1:
        count += 1
print((count/len(train_data))*100)

17.46


In [27]:
list(vae2(ex_done.to("cuda")))[0][10]

tensor(0.7629, device='cuda:0', grad_fn=<SelectBackward>)

In [28]:
list(vae2(ex_notdone.to("cuda")))[0][10]

tensor(0.0045, device='cuda:0', grad_fn=<SelectBackward>)

In [29]:
from torch.autograd import Variable

In [30]:
sample = Variable(torch.randn(2, LATENT_SIZE))

In [31]:
res = vae2.decode(sample.to("cuda"))

In [32]:
res

tensor([[0.5005, 0.4993, 0.4999, 0.4994, 0.4881, 0.4986, 0.5009, 0.4987, 0.5089,
         0.0523, 0.0026, 0.0027],
        [0.5011, 0.4997, 0.5001, 0.5005, 0.5171, 0.4965, 0.4988, 0.5035, 0.4967,
         0.0554, 0.9997, 0.9997]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [33]:
res = [0, 0, 0]
k = []
for i in range(10000):
    sample = Variable(torch.randn(1, LATENT_SIZE))
    a = vae1.decode(sample.to("cuda"))[0]
    num = float(sum(list(a[10:])))
    k.append(num)
    if num < 0.2:
        res[0] += 1
    elif num <= 0.7:
        res[1] += 1
    else:
        res[2] += 1
        
d = 0
nd = 0

for a in test_data:
    if a[10] == 1:
        d += 1
    else:
        nd += 1
        
        
print(f"UNBALANCED DATASET")
print(f"counts of intervals: [0, 0.1]: {res[0]}, [0.1, 0.7]: {res[1]}, [0.7, 1]: {res[2]}")
print(f"-- so the ratio to done and not done (deleting the middle ones) is {res[2]/(res[2]+res[0])}")
print(f"The real ration is {d/(nd+d)}")

UNBALANCED DATASET
counts of intervals: [0, 0.1]: 7686, [0.1, 0.7]: 587, [0.7, 1]: 1727
-- so the ratio to done and not done (deleting the middle ones) is 0.18346966960586422
The real ration is 0.17494


In [34]:
import gym
env = gym.make("InvertedPendulum-v2")
env.reset()
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0] 
action_low = env.action_space.low[0]
action_high = env.action_space.high[0]
state_low = -10.0
state_high = 10.0

In [35]:
def descale(x):
    # State
    ((x[:, 0].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 1].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 2].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 3].mul_(state_high-state_low)).add_(state_low)).to(device)

    # Action
    ((x[:, 4].mul_(action_high-action_low)).add_(action_low)).to(device)
    
    # State
    ((x[:, 5].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 6].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 7].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 8].mul_(state_high-state_low)).add_(state_low)).to(device)
    
    # Reward
    (x[:, 9].mul_(20.0)).to(device)
    
    # Done
    (x[:, 10].round_()).to(device)
    
    return x

In [36]:
sample = Variable(torch.randn(2, LATENT_SIZE)).cuda()
x = vae1.decode(sample)
x[0][10]

tensor(0.0026, device='cuda:0', grad_fn=<SelectBackward>)

In [37]:
x = descale(x)

In [38]:
x[0][10]

tensor(0., device='cuda:0', grad_fn=<SelectBackward>)